# $\pi$ by means of the arithmetic-geometric mean

The mathematical constant $\pi$ can be determined by means of the arithmetic-geometric mean as demonstrated in [E. Salamin, Math. Comp. **30**, 565 (1976)](https://doi.org/10.2307%2F2005327). Theorem 1b in that paper states that
$$\pi = \frac{4[\mathrm{agm}(1, 2^{-1/2})]^2}{ 1-\sum_{n=1}^\infty 2^{n+1}c_n^2}\,.$$
To introduce the arithmetic-geometric mean $\mathrm{agm}(a, b)$ of $a$ and $b$ and the numbers $c_n$ appearing in this relation, we consider the recursion
$$a_{n+1} = \frac{a_n+b_n}{2}\,,\quad b_{n+1} = \sqrt{a_{n-1}b_{n-1}}$$
with the initial values $a_0=a, b_0=b$. For $n\rightarrow\infty$, $a_n$ and $b_n$ converge to the same number, the arithmetic-geometric mean of $a$ and $b$. Furthermore, we define
$$c_n^2=a_n^2-b_n^2\,.$$

In a first step, an implementation of this prescription using floats shall be developed. In this way, we will familiarize ourselves with the computation of $\pi$ by means of the arithmetic-geometric mean. Ideally, the iteration is carried out until $c_n$ becomes zero. However, rounding errors might prevent the series $(a_n, b_n)$ reach a point where $a_n=b_n$ within floating point precision.

In a second step, the calculation should be done by means of integers thus taking advantage of the fact that in Python integers can become sufficiently large. This approach will allow us to determine $\pi$ to a large number of digits.

## Part 1: Implementation with floats

Let us start by getting acquainted with the arithmetic-geometric mean. Implement a function which takes $a_n$ and $b_n$ as arguments and returns $a_{n+1}$, $b_{n+1}$, as well as $a_{n+1}^2-b_{n+1}^2$.

In [ ]:
# if needed, modules should be imported before functions are defined
# BEGIN SOLUTION
from math import sqrt
# END SOLUTION

def agm_iteration(a, b):
    """perform one arithmetic-geometric iteration
    
       This function takes two floats a and b as
       a_n and b_n and returns a_{n+1}, b_{n+1},
       and the difference of their squares.
    """
    # BEGIN SOLUTION
    a, b = (a+b)/2, sqrt(a*b)
    c2 = a*a-b*b
    return a, b, c2
    # END SOLUTION
    

Execute the following two cells to test your code.

In [ ]:
try:
    agm_iteration(1, 1)
except NameError as e:
    if 'sqrt' in repr(e):
        raise AssertionError('Did you forget to import a required module?')
assert agm_iteration(1, 1) is not None, 'Did you return a result?'
assert len(agm_iteration(1, 1)) == 3, 'It seems that you are not returning three values.'

In [ ]:
import random
from math import sqrt
x = random.random()
assert agm_iteration(x, x) == (x, x, 0), 'There is a problem even for the trivial case of equal arguments.'
assert agm_iteration(x, 0) == (0.5*x, 0, 0.25*x**2), 'There is a problem if one of the arguments is zero.'
y = random.random()
assert agm_iteration(x, y)[1] != sqrt(0.5*(x+y)*y), ('Make sure that you do not accidentally change the '
                                                     'variable a before you evaluate the square root.')
assert agm_iteration(x, y)[:-1] == (0.5*(x+y), sqrt(x*y)), 'There is an error in the new values of a and b.'
assert abs(agm_iteration(x, y)[2]-0.25*(x-y)**2) < 1e-14, 'The difference of the squares is not correct.'

Now, you can determine $\pi$ by means of the arithmetic-geometric mean. Since at this point floats are used, rounding errors will prevent you from obtaining the optimal result. You should aim for an relative error with respect to the correct result smaller than $2\cdot10^{-14}$.

In [1]:
# if needed, modules should be imported before functions are defined
# BEGIN SOLUTION
from math import sqrt
# END SOLUTION

def pi_agm_float():
    """determine pi by means of the arithmetic-geometric mean
    
    """
    # BEGIN SOLUTION
    a = 1
    b = 1/sqrt(2)
    c_sum = 0
    faktor_two = 2
    c2 = 1

    while c2 > 1e-15:
        a, b = (a+b)/2, sqrt(a*b)
        c2 = a*a-b*b
        faktor_two = faktor_two*2
        c_sum = c_sum+faktor_two*c2

    return 4*a**2/(1-c_sum)
    # END SOLUTION

By executing the following two cells, you can verify that your code runs correctly.

In [2]:
try:
    result = pi_agm_float()
except NameError as e:
    if 'sqrt' in repr(e):
        raise AssertionError('Did you forget to import a required module?')
assert result is not None, 'Did you return a result?'
assert type(result) == type(1.), 'The result should be a float.'

In [4]:
from math import pi
result = pi_agm_float()
relerror = abs(result-pi)/pi
assert relerror <= 2e-14, 'The relative error {:.3g} is larger than allowed.'.format(relerror)
assert relerror != 0., 'Did you simply return the value of π from the math module?'

## Part 2: Implementation with integers

A much higher precision can be achieved by doing the complete calculation in integer arithmetic after expanding the fraction by a more or less large power of ten.

As a first step, we need to be able to determine the square root of a large integer. A straightforward way to do so is by means of [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method). In order to reduce the compute time, the square root of the `math`-module can be used to obtain a good estimate as a starting point.

Explain why the `sqrt`-function of the `math`-module cannot be used to take the square root of a large integer.

`sqrt` will first try to convert the integer argument into a float. This attempt will fail if the integer is larger than the largest possible float.